In [ ]:
from neo4j import GraphDatabase

# Connect to the Neo4j database
driver = GraphDatabase.driver("bolt://neo4j-bolt.monarchinitiative.org:7687")

# Define the Cypher query
query = """
MATCH
  (upheno:`biolink:PhenotypicFeature` WHERE upheno.id STARTS WITH "UPHENO:")<-[:`biolink:subclass_of`]-(mouse_pheno:`biolink:PhenotypicFeature`)<-[:`biolink:has_phenotype`]-(mouse_gene:`biolink:Gene` where "NCBITaxon:10090" in mouse_gene.in_taxon)-[:`biolink:orthologous_to`]-(human_gene:`biolink:Gene` where "NCBITaxon:9606" in human_gene.in_taxon)
RETURN upheno.id, mouse_pheno.id, mouse_gene.id, human_gene.id
"""

# Run the query and print the results
with driver.session() as session:
    results = session.run(query)
    for record in results:
        print('\t'.join(record))